# Spatial changes

This notebook is made to compute & display some spatial changes btw two images (single dates or composite using different temporal aggregation frequency)

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [3]:
import ee, geemap,eemont
import pandas as pd

import sys
sys.path.append('../preprocessing/clouds')
import landsat_preprocessing
import s2_preprocessing

import sys
sys.path.append('../wetlands_detection')
import wetlands_unsupervised_clustering

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()


Successfully saved authorization token.


In [4]:
roi = ee.Geometry.Polygon([[[17.204933, 60.402663], [17.204933, 60.455525], [17.2645, 60.455525], [17.2645, 60.402663], [17.204933, 60.402663]]])

In [6]:
def filter_col(col, roi, band, thresh):
    
    col = col.map(lambda image : image.clip(roi))

    def count_pixels(image,roi): 
        pixel_count = image.select(band).reduceRegion(
            reducer= ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get(band)
        return image.set('pixel_count', pixel_count)

    nb_pixels_ts = col.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_ts.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * thresh

    # Filter the collection based on the pixel count threshold
    filtered_col = nb_pixels_ts.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_col

def get_gd_cloud_free_col(roi, thresh):
    
    gd = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(roi)
    gd = filter_col(gd,roi,'label',thresh)
    
    return gd

def add_dates(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return image.set({'date': date})

In [22]:
gd = get_gd_cloud_free_col(roi, 0.3)
img_1 = gd.filterDate('2017-05-01', '2017-09-30').select('label').mode()
img_2 = gd.filterDate('2018-05-01', '2018-09-30').select('label').mode()
s2_img_1 = s2_preprocessing.get_s2_cloud_free_col_dates(roi,'2017-05-01', '2017-09-30',0.3).median()
#s2_img_2 = s2_preprocessing.get_s2_cloud_free_col_dates(roi,'2018-05-01','2018-05-01',0.3).median()

EEException: Collection.first: Empty date ranges not supported for the current operation.

In [ ]:
min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')

wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)

map = geemap.Map()

dw_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419bdf',  # water
        '#397d49',  # trees
        '#88b053',  # grass
        '#7a87c6',  # flooded_vegetation
        '#e49635',  # crops
        '#dfc35a',  # shrub_and_scrub
        '#c4281b',  # built
        '#a59b8f',  # bare
        '#b39fe1',  # snow_and_ice
    ],
}

# Legend for Dynamic World
legend_dw = {
    'Water': '#419bdf',
    'Trees': '#397d49',
    'Grass': '#88b053',
    'Flooded Vegetation': '#7a87c6',
    'Crops': '#e49635',
    'Shrub and Scrub': '#dfc35a',
    'Built': '#c4281b',
    'Bare': '#a59b8f',
    'Snow and Ice': '#b39fe1',
}

map.centerObject(roi, 14)
# Create and add the time slider with appropriate number of labels
map.addLayer(img_1.select('label'), dw_vis, 'cls_1')
map.addLayer(img_2.select('label'), dw_vis, 'cls_2')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
map.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
condition_neg = (img_1.eq(2).Or(img_1.eq(3))).And(img_2.eq(5))
# Mask the image to keep only the relevant pixels
change_detection_neg = condition_neg.selfMask()

condition_pos = img_1.eq(5).And((img_2.eq(2)).Or(img_2.eq(3)))
change_detection_pos = condition_pos.selfMask()

# Add the change detection layer to the map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(s2_img_1.divide(10000), {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 0.1},'s2_img' )
m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
m.addLayer(change_detection_neg, {'palette':['red']}, 'negative changes')
m.addLayer(change_detection_pos, {'palette':['green']}, 'positive changes')
# Display the map
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…